# Air Quality Analysis in Victoria
---------------------------------
This notebook explores air quality data collected from various monitoring stations in Victoria. The dataset includes hourly average measurements of air quality parameters. The objective of this analysis is to gain insights into the temporal trends, spatial variations, and potential impacts of air pollution on public health in Victoria.

## Import Necessary Libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

## Load the Dataset

### Dataset Description:
--------------------

The dataset used in this analysis contains hourly average measurements of air quality parameters from various monitoring stations in Victoria. The dataset is sourced from EPA and includes the following columns:

- date: Date of observation
- time: Time of observation
- location_name: Name of the monitoring station
- latitude: Latitude coordinates of the monitoring station
- longitude: Longitude coordinates of the monitoring station
- value: Value of the observed air quality parameter
- parameter_name: Name of the observed air quality parameter

The objective of this analysis is to explore the relationship between air quality parameters and potential impacts on public health, particularly focusing on parameters such as CO (Carbon Monoxide), PM10 (Particulate Matter with diameter less than 10 micrometers), PM2.5 (Particulate Matter with diameter less than 2.5 micrometers), O3 (Ozone), and SO2 (Sulfur Dioxide). These parameters are known to have significant implications for air quality and human health.

Before proceeding with the analysis, we will load the dataset.


In [6]:
# Set display options for pandas DataFrame
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', 1000)        # Set display width

In [12]:
# Load the dataset
data = pd.read_excel('../data/2022_All_sites_air_quality_hourly_avg.xlsx', sheet_name='AllData', usecols='A:G')

# Filter the DataFrame to select only specific parameter names
selected_parameters = ['CO', 'PM10', 'PM2.5', 'O3', 'SO2']
filtered_data = data[data['parameter_name'].isin(selected_parameters)]

## Data Preprocessing:
-------------------

Before delving into detailed analysis, it's crucial to preprocess the dataset to ensure its quality and suitability for further exploration:

1. Data Conversion: Convert the 'date' column to datetime format to facilitate temporal analysis.
2. Filtering: Apply filters to select specific air quality parameters of interest for focused analysis.

By performing these preprocessing steps, we can streamline the data and focus our analysis on the relevant air quality parameters, setting the foundation for insightful exploration.


In [18]:
# Reset the index
filtered_data.reset_index(drop=True, inplace=True)

# Convert 'date' column to datetime data type
filtered_data['date'] = pd.to_datetime(filtered_data['date'])

# Extract only the date portion from the datetime column
filtered_data['date'] = filtered_data['date'].dt.date

/var/folders/7p/m7gs_hkn58qbs70h8pztg9480000gn/T/ipykernel_12001/1248701647.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['date'] = pd.to_datetime(filtered_data['date'])
/var/folders/7p/m7gs_hkn58qbs70h8pztg9480000gn/T/ipykernel_12001/1248701647.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['date'] = filtered_data['date'].dt.date


In [19]:
# Display the first few rows of the filtered DataFrame
print(filtered_data.head())

         date      time location_name   latitude   longitude   value parameter_name
0  2021-12-31  23:00:00  Morwell East -38.229393  146.424454   0.356             CO
1  2021-12-31  23:00:00   Mooroolbark -37.774967  145.328500  49.150           PM10
2  2021-12-31  23:00:00     Traralgon -38.194282  146.531464  27.504           PM10
3  2021-12-31  23:00:00     Footscray -37.803709  144.869342  33.061           PM10
4  2021-12-31  23:00:00  Morwell East -38.229393  146.424454   3.327          PM2.5
